In [1]:
from netCDF4 import Dataset
import xarray as xr
import dask
import os
import requests


#Allows us to visualize the dask progress for parallel operations
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [2]:
import urllib
from urllib import request
from http.cookiejar import CookieJar
import json
from urllib import request, parse
from http.cookiejar import CookieJar
import getpass
import netrc



def get_token( url: str,client_id: str, user_ip: str,endpoint: str) -> str:
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
        token: str = ''
        xml: str = """<?xml version='1.0' encoding='utf-8'?>
        <token><username>{}</username><password>{}</password><client_id>{}</client_id>
        <user_ip_address>{}</user_ip_address></token>""".format(username, password, client_id, user_ip)
        headers: Dict = {'Content-Type': 'application/xml','Accept': 'application/json'}
        resp = requests.post(url, headers=headers, data=xml)
        response_content: Dict = json.loads(resp.content)
        token = response_content['token']['id']
    except:
        print("Error getting the token - check user name and password")
    return token

def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file
    first and if no credentials are found, it prompts for them.
    Valid endpoints include:
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    
    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

edl="urs.earthdata.nasa.gov"
cmr="cmr.earthdata.nasa.gov"

setup_earthdata_login_auth(edl)
token_url="https://"+cmr+"/legacy-services/rest/tokens"
token=get_token(token_url,'podaac-subscriber', "127.0.0.1",edl)

In [6]:
r = requests.get('https://cmr.earthdata.nasa.gov/search/granules.umm_json?collection_concept_id=C1968979597-POCLOUD&temporal=2021-11-01T10:00:00Z,2021-11-02T00:00:00Z&pageSize=365&token='+ token)
response_body = r.json()

In [7]:
od_files = []
for itm in response_body['items']:
    for urls in itm['umm']['RelatedUrls']:
        if 'OPeNDAP' in urls['Description']:
            od_files.append(urls['URL'])

od_files
        

['https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_043_20211101T093621_20211101T111451_F03',
 'https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_045_20211101T111451_20211101T130809_F03',
 'https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_047_20211101T130809_20211101T150357_F03',
 'https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_049_20211101T150357_20211101T170013_F03',
 'https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_051_20211101T170013_20211101T185611_F03',
 'https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_053_20211101T185611_20211101T205202_F03',
 'https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_056_20211101T205202_2021110

In [8]:
len(od_files)

8

In [10]:
for f in od_files:
    print (f)
    data_url = f'{f}.dap.nc4'
    
    
    

https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_043_20211101T093621_20211101T111451_F03
https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_045_20211101T111451_20211101T130809_F03
https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_047_20211101T130809_20211101T150357_F03
https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_049_20211101T150357_20211101T170013_F03
https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_051_20211101T170013_20211101T185611_F03
https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_053_20211101T185611_20211101T205202_F03
https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_056_20211101T205202_20211101T223040_F03
https://opend

In [12]:
data_url

'https://opendap.earthdata.nasa.gov/collections/C1968979597-POCLOUD/granules/S6A_P4_2__LR_STD__NR_036_057_20211101T223040_20211102T002212_F03.dap.nc4'